In [1]:
import pandas as pd

## Accuracy

### Eval Categorical

In [3]:
df = pd.read_csv("sim_base_to_onnx_cat.csv")

In [4]:
# df.head(5)
df.keys()

Index(['skill', 'reader', 'adapter', 'timestamp', 'answer_base',
       'logits_answer_base', 'answer_quantized_model',
       'logits_answer_quantized_model', 'answer_onnx_model',
       'logits_answer_onnx_model', 'answer_onnx_opt_model',
       'logits_answer_onnx_opt_model', 'answer_quant_onnx_model',
       'logits_answer_quant_onnx_model', 'answer_quant_onnx_opt_model',
       'logits_answer_quant_onnx_opt_model', 'data_id', 'dataset', 'question',
       'context', 'answer_dataset'],
      dtype='object')

In [5]:
cat_df = df[df["skill"] == "categorical"]

In [18]:
def get_mean_diff_of_logits(datafr, model_name):
    l = datafr[model_name].str.replace("([\[\]])", "", regex=True)

    float_list = []
    for v in l:
        s = v.split()
        float_list.append([float(s[0]), float(s[1])])

    r_list = []
    for fe in float_list:
        r = fe[0] - fe[1]
        r_list.append(r)
        
    mean_diff = sum(r_list) / len(r_list)
    return abs(mean_diff) # return only pos. value

#### Compare to gold label

In [6]:
gold_label_column_name = "answer_dataset"
model_name_answer_column_list = ["answer_base", "answer_quantized_model", "answer_onnx_model", "answer_onnx_opt_model", "answer_quant_onnx_model", "answer_quant_onnx_opt_model"]
model_name_logits_column_list = ["logits_answer_base", "logits_answer_quantized_model", "logits_answer_onnx_model", "logits_answer_onnx_opt_model", "logits_answer_quant_onnx_model", "logits_answer_quant_onnx_opt_model"]

In [7]:
for adapter in cat_df["adapter"].unique():
    adapter_cat_df = cat_df[cat_df["adapter"] == adapter]
    for reader in adapter_cat_df["reader"].unique():
        reader_adapter_cat_df = adapter_cat_df[adapter_cat_df["reader"] == reader].reset_index()
        print(f"Loading: {reader} {adapter}")

        total_amount = len(reader_adapter_cat_df)
        for column_name in model_name_answer_column_list:
            t = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] == reader_adapter_cat_df[column_name]]
            f = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] != reader_adapter_cat_df[column_name]]

            total_true = len(t)
            total_false = len(f)

            accuracy = (total_true)/total_amount

            print(f"Accuracy for {reader} {adapter} {column_name} is: {accuracy}. T: {total_true}. F: {total_false}.")


Loading: bert-base-uncased boolq
Accuracy for bert-base-uncased boolq answer_base is: 0.7339449541284404. T: 2400. F: 870.
Accuracy for bert-base-uncased boolq answer_quantized_model is: 0.7363914373088685. T: 2408. F: 862.
Accuracy for bert-base-uncased boolq answer_onnx_model is: 0.6174311926605505. T: 2019. F: 1251.
Accuracy for bert-base-uncased boolq answer_onnx_opt_model is: 0.6174311926605505. T: 2019. F: 1251.
Accuracy for bert-base-uncased boolq answer_quant_onnx_model is: 0.5376146788990825. T: 1758. F: 1512.
Accuracy for bert-base-uncased boolq answer_quant_onnx_opt_model is: 0.5376146788990825. T: 1758. F: 1512.
Loading: roberta-base boolq
Accuracy for roberta-base boolq answer_base is: 0.789908256880734. T: 2583. F: 687.
Accuracy for roberta-base boolq answer_quantized_model is: 0.791743119266055. T: 2589. F: 681.
Accuracy for roberta-base boolq answer_onnx_model is: 0.7660550458715596. T: 2505. F: 765.
Accuracy for roberta-base boolq answer_onnx_opt_model is: 0.7660550458

In [8]:
# Calc diff of logits (of binary) if pred is wrong.

In [17]:
# Testing

# t = df.index[df["answer_base"] == df["answer_onnx_model"]]
# f = df.index[df["answer_base"] != df["answer_onnx_model"]]

# false_onnx = df.iloc[f]
# true_onnx = df.iloc[t]

# #get result for false results for onnx model
# model_name_logits = model_name_logits_column_list[2]
# mean_diff_false_results = get_mean_diff_of_logits(false_onnx, model_name_logits)
# print(mean_diff_false_results)
# mean_diff_true_results = get_mean_diff_of_logits(true_onnx, model_name_logits)
# print(mean_diff_true_results)
# mean_diff_all_results = get_mean_diff_of_logits(df, model_name_logits)
# print(mean_diff_all_results)

In [13]:
for adapter in cat_df["adapter"].unique():
    adapter_cat_df = cat_df[cat_df["adapter"] == adapter]
    for reader in adapter_cat_df["reader"].unique():
        reader_adapter_cat_df = adapter_cat_df[adapter_cat_df["reader"] == reader].reset_index()
        print(f"Loading: {reader} {adapter}")
        
        for column_name in model_name_answer_column_list:
            t = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] == reader_adapter_cat_df[column_name]]
            f = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] != reader_adapter_cat_df[column_name]]

            column_name_logits = model_name_logits_column_list[model_name_answer_column_list.index(column_name)]
            false_pred_df = reader_adapter_cat_df.iloc[f]
            true_pred_df = reader_adapter_cat_df.iloc[t]

            mean_diff_false_results = get_mean_diff_of_logits(false_pred_df, column_name_logits)
            mean_diff_true_results = get_mean_diff_of_logits(true_pred_df, column_name_logits)
            print(f"{reader} {adapter}. Mean diff logits wrong answer: {column_name_logits} {mean_diff_false_results}")
            print(f"{reader} {adapter}. Mean diff logits right answer: {column_name_logits} {mean_diff_true_results}")

            print("______\n")

Loading: bert-base-uncased boolq
bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_base 0.5527182899770107
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_base 1.2831902399541695
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_quantized_model 0.5779924959446868
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_quantized_model 1.2983542962500025
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_onnx_model 0.10977075498867121
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_onnx_model 0.7066019500930119
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_onnx_opt_model 0.10977075498867121
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_onnx_opt_model 0.7066019500930119
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_quant_onnx_model 0.5516529688425929
bert-base-uncased boo

#### Compare to Base pred

In [15]:
gold_label_column_name = "answer_base"
model_name_answer_column_list = ["answer_quantized_model", "answer_onnx_model", "answer_onnx_opt_model", "answer_quant_onnx_model", "answer_quant_onnx_opt_model"]
model_name_logits_column_list = ["logits_answer_quantized_model", "logits_answer_onnx_model", "logits_answer_onnx_opt_model", "logits_answer_quant_onnx_model", "logits_answer_quant_onnx_opt_model"]

In [16]:
for adapter in cat_df["adapter"].unique():
    adapter_cat_df = cat_df[cat_df["adapter"] == adapter]
    for reader in adapter_cat_df["reader"].unique():
        reader_adapter_cat_df = adapter_cat_df[adapter_cat_df["reader"] == reader].reset_index()
        print(f"Loading: {reader} {adapter}")

        total_amount = len(reader_adapter_cat_df)
        for column_name in model_name_answer_column_list:
            t = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] == reader_adapter_cat_df[column_name]]
            f = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] != reader_adapter_cat_df[column_name]]

            total_true = len(t)
            total_false = len(f)

            accuracy = (total_true)/total_amount

            print(f"Accuracy for {reader} {adapter} {column_name} is: {accuracy}. T: {total_true}. F: {total_false}.")


Loading: bert-base-uncased boolq
Accuracy for bert-base-uncased boolq answer_quantized_model is: 0.9914373088685016. T: 3242. F: 28.
Accuracy for bert-base-uncased boolq answer_onnx_model is: 0.7091743119266055. T: 2319. F: 951.
Accuracy for bert-base-uncased boolq answer_onnx_opt_model is: 0.7091743119266055. T: 2319. F: 951.
Accuracy for bert-base-uncased boolq answer_quant_onnx_model is: 0.5565749235474006. T: 1820. F: 1450.
Accuracy for bert-base-uncased boolq answer_quant_onnx_opt_model is: 0.5565749235474006. T: 1820. F: 1450.
Loading: roberta-base boolq
Accuracy for roberta-base boolq answer_quantized_model is: 0.9877675840978594. T: 3230. F: 40.
Accuracy for roberta-base boolq answer_onnx_model is: 0.8746177370030581. T: 2860. F: 410.
Accuracy for roberta-base boolq answer_onnx_opt_model is: 0.8746177370030581. T: 2860. F: 410.
Accuracy for roberta-base boolq answer_quant_onnx_model is: 0.8486238532110092. T: 2775. F: 495.
Accuracy for roberta-base boolq answer_quant_onnx_opt_m

In [19]:
for adapter in cat_df["adapter"].unique():
    adapter_cat_df = cat_df[cat_df["adapter"] == adapter]
    for reader in adapter_cat_df["reader"].unique():
        reader_adapter_cat_df = adapter_cat_df[adapter_cat_df["reader"] == reader].reset_index()
        print(f"Loading: {reader} {adapter}")
        
        for column_name in model_name_answer_column_list:
            t = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] == reader_adapter_cat_df[column_name]]
            f = reader_adapter_cat_df.index[reader_adapter_cat_df[gold_label_column_name] != reader_adapter_cat_df[column_name]]

            column_name_logits = model_name_logits_column_list[model_name_answer_column_list.index(column_name)]
            false_pred_df = reader_adapter_cat_df.iloc[f]
            true_pred_df = reader_adapter_cat_df.iloc[t]

            mean_diff_false_results = get_mean_diff_of_logits(false_pred_df, column_name_logits)
            mean_diff_true_results = get_mean_diff_of_logits(true_pred_df, column_name_logits)
            print(f"{reader} {adapter}. Mean diff logits wrong answer: {column_name_logits} {mean_diff_false_results}")
            print(f"{reader} {adapter}. Mean diff logits right answer: {column_name_logits} {mean_diff_true_results}")

            print("______\n")

Loading: bert-base-uncased boolq
bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_quantized_model 0.011839845357142859
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_quantized_model 1.1179318819260728
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_onnx_model 0.1826791926197525
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_onnx_model 0.7493231840922832
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_onnx_opt_model 0.1826791926197525
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_onnx_opt_model 0.7493231840922832
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_quant_onnx_model 0.6219549973655173
bert-base-uncased boolq. Mean diff logits right answer: logits_answer_quant_onnx_model 0.21897178857687236
______

bert-base-uncased boolq. Mean diff logits wrong answer: logits_answer_quant_onnx_opt_model 0.621954997

### Eval Mcq

In [20]:
df = pd.read_csv("sim_base_to_onnx_mcq.csv")
mcq_df = df[df["skill"] == "multiple-choice"]

In [21]:
mcq_df.keys()

Index(['skill', 'reader', 'adapter', 'timestamp', 'answer_base',
       'logits_answer_base', 'answer_quantized_model',
       'logits_answer_quantized_model', 'answer_onnx_model',
       'logits_answer_onnx_model', 'answer_onnx_opt_model',
       'logits_answer_onnx_opt_model', 'answer_quant_onnx_model',
       'logits_answer_quant_onnx_model', 'answer_quant_onnx_opt_model',
       'logits_answer_quant_onnx_opt_model', 'data_id', 'dataset', 'question',
       'context', 'choices', 'answer_dataset'],
      dtype='object')

In [22]:
# mcq_df

In [23]:
gold_label_column_name = "answer_dataset"
model_name_answer_column_list = ["answer_base", "answer_quantized_model", "answer_onnx_model", "answer_onnx_opt_model", "answer_quant_onnx_model", "answer_quant_onnx_opt_model"]
model_name_logits_column_list = ["logits_answer_base", "logits_answer_quantized_model", "logits_answer_onnx_model", "logits_answer_onnx_opt_model", "logits_answer_quant_onnx_model", "logits_answer_quant_onnx_opt_model"]

In [26]:
for adapter in mcq_df["adapter"].unique():
    adapter_mcq_df = mcq_df[mcq_df["adapter"] == adapter]
    for reader in adapter_mcq_df["reader"].unique():
        print(f"Loading: {reader} {adapter}")
        reader_adapter_mcq_df = adapter_mcq_df[adapter_mcq_df["reader"] == reader].reset_index()

        total_amount = len(reader_adapter_mcq_df)
        for column_name in model_name_answer_column_list:
            t = reader_adapter_mcq_df.index[reader_adapter_mcq_df["answer_dataset"] == reader_adapter_mcq_df[column_name]]
            f = reader_adapter_mcq_df.index[reader_adapter_mcq_df["answer_dataset"] != reader_adapter_mcq_df[column_name]]

            total_true = len(t)
            total_false = len(f)

            accuracy = (total_true)/total_amount

            print(f"Accuracy for {reader} {adapter} {column_name} is: {accuracy}. T: {total_true}. F: {total_false}.")


Loading: bert-base-uncased cosmos_qa
Accuracy for bert-base-uncased cosmos_qa answer_base is: 0.6050251256281407. T: 1806. F: 1179.
Accuracy for bert-base-uncased cosmos_qa answer_quantized_model is: 0.6053601340033501. T: 1807. F: 1178.
Accuracy for bert-base-uncased cosmos_qa answer_onnx_model is: 0.5149078726968174. T: 1537. F: 1448.
Accuracy for bert-base-uncased cosmos_qa answer_onnx_opt_model is: 0.5149078726968174. T: 1537. F: 1448.
Accuracy for bert-base-uncased cosmos_qa answer_quant_onnx_model is: 0.4123953098827471. T: 1231. F: 1754.
Accuracy for bert-base-uncased cosmos_qa answer_quant_onnx_opt_model is: 0.4123953098827471. T: 1231. F: 1754.
Loading: roberta-base cosmos_qa
Accuracy for roberta-base cosmos_qa answer_base is: 0.7078726968174205. T: 2113. F: 872.
Accuracy for roberta-base cosmos_qa answer_quantized_model is: 0.7065326633165829. T: 2109. F: 876.
Accuracy for roberta-base cosmos_qa answer_onnx_model is: 0.6100502512562814. T: 1821. F: 1164.
Accuracy for roberta-

In [27]:
reader_adapter_mcq_df

,index,skill,reader,adapter,timestamp,answer_base,logits_answer_base,answer_quantized_model,logits_answer_quantized_model,answer_onnx_model,...,answer_quant_onnx_model,logits_answer_quant_onnx_model,answer_quant_onnx_opt_model,logits_answer_quant_onnx_opt_model,data_id,dataset,question,context,choices,answer_dataset
0,8518,multiple-choice,roberta-base,quartz,2023-01-23 14:14:36.334767,decrease,[[ 1.5967226 -1.6974251]],decrease,[[ 1.5605446 -1.6843979]],decrease,...,decrease,[[-1.0959533 -2.387991 ]],decrease,[[-1.0959533 -2.387991 ]],QRQA-10372-1-flip,quartz,If Jim moves some particles of matter farther ...,"When particles of matter are closer together, ...","['decrease', 'increase']",decrease
1,8519,multiple-choice,roberta-base,quartz,2023-01-23 14:14:36.664558,increased,[[-1.4991442 -1.7471755]],increased,[[-1.4956504 -1.7537297]],decreased,...,decreased,[[-2.2649467 -1.5029308]],decreased,[[-2.2649467 -1.5029308]],QRQA-10371-4-flip,quartz,Long ago the surface of Venus warmed enough th...,An increase in greenhouse gases leads to great...,"['increased', 'decreased']",decreased
2,8520,multiple-choice,roberta-base,quartz,2023-01-23 14:14:37.048579,less,[[-0.04023071 1.9518709 ]],less,[[-0.07223731 1.977018 ]],less,...,less,[[-0.7888222 -0.02593471]],less,[[-0.7888222 -0.02593471]],QRQA-10296-1-flip,quartz,If less waters falls on an area of land it wil...,"As more water covered the land, sand and silt ...","['more', 'less']",less
3,8521,multiple-choice,roberta-base,quartz,2023-01-23 14:14:37.443374,open,[[1.9225118 3.9954627]],open,[[2.0977662 3.9687064]],open,...,open,[[3.340151 3.5526862]],open,[[3.340151 3.5526862]],QRQA-10115-3,quartz,Rich applies a solution to the dish that incre...,The increase in turgor pressure of the guard c...,"['close', 'open']",open
4,8522,multiple-choice,roberta-base,quartz,2023-01-23 14:14:37.793810,older,[[ 0.64913607 -0.49488717]],older,[[ 0.67850906 -0.53324205]],older,...,older,[[ 0.42830256 -0.4287805 ]],older,[[ 0.42830256 -0.4287805 ]],QRQA-10082-1,quartz,Simon was digging in his yard and found that t...,"Therefore, deeper rock layers must be older th...","['older', 'younger']",older
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,8897,multiple-choice,roberta-base,quartz,2023-01-23 14:17:19.857484,Mercury,[[1.6974237 1.4413342]],Mercury,[[1.7165381 1.5000093]],Mars,...,Mercury,[[-1.6023277 -1.7910004]],Mercury,[[-1.6023277 -1.7910004]],QRQA-10106-5-flip,quartz,Which planet has the least gravity exerted on ...,Objects that are closer together have a strong...,"['Mercury', 'Mars']",Mars
380,8898,multiple-choice,roberta-base,quartz,2023-01-23 14:17:20.281529,less,[[-1.1857404 2.4406533]],less,[[-1.27052 2.5250819]],less,...,less,[[-0.47397986 0.09450573]],less,[[-0.47397986 0.09450573]],QRQA-10345-2,quartz,When you get further from the sun are planets ...,When valence electrons are farther from the nu...,"['more', 'less']",less
381,8899,multiple-choice,roberta-base,quartz,2023-01-23 14:17:20.749195,larger,[[ 1.1175296 -0.8436286]],larger,[[ 1.2071087 -0.8636759]],larger,...,larger,[[ 1.0533252 -1.0935922]],larger,[[ 1.0533252 -1.0935922]],QRQA-10357-4,quartz,Don is watching rocks be eroded by water. Sinc...,"In erosion, the more energy the water has, the...","['larger', 'smaller']",larger
382,8900,multiple-choice,roberta-base,quartz,2023-01-23 14:17:21.178662,increases,[[ 3.5217798 -0.75384456]],increases,[[ 3.4947906 -0.75599843]],increases,...,increases,[[2.5232887 0.26663825]],increases,[[2.5232887 0.26663825]],QRQA-10257-1,quartz,If Milo is growing some plants in his garden a...,Increases the sheen on leaves to make them mor...,"['increases', 'decreases']",increases


In [ ]:
model_name = model_name_logits_column_list[2]

l = datafr[model_name].str.replace("([\[\]])", "", regex=True)

In [ ]:
def get_mean_diff_of_logits(datafr, model_name):
    l = datafr[model_name].str.replace("([\[\]])", "", regex=True)

    float_list = []
    for v in l:
        s = v.split()
        float_list.append([float(s[0]), float(s[1])])

    r_list = []
    for fe in float_list:
        r = fe[0] - fe[1]
        r_list.append(r)
        
    mean_diff = sum(r_list) / len(r_list)
    return abs(mean_diff) # return only pos. value

## Inference Time

In [169]:
df = pd.read_csv("inference_time_categorical.csv")
# df = pd.read_csv("inference_time_extractive.csv")
# df = pd.read_csv("")

In [171]:
df.head()

,Unnamed: 0,adapter,reader,model_name,mean_time,median_time,min_time,max_time,mean_time_per_token,median_time_per_token,min_time_per_token,max_time_per_token,runs,time_unique_values,seq_length,context,question,data_id
0,0,drop,bert-base-uncased,Base,129.511690,116.297960,100.482702,190.842867,0.824915,0.740751,100.482702,190.842867,5,"[116.29796028137208, 190.842866897583, 108.004...",157,Hoping to rebound from their loss to the Patr...,Who scored the first touchdown of the game?,f37e81fa-ef7b-4583-b671-762fc433faa9
1,1,drop,bert-base-uncased,Base,118.036652,101.804018,88.285923,202.951193,0.751826,0.648433,88.285923,202.951193,5,"[106.0810089111328, 202.95119285583496, 88.285...",157,Hoping to rebound from their loss to the Patr...,How many field goals did Kris Brown kick?,ac6ba235-3024-4f63-a6ab-730a14def4cb
2,2,drop,bert-base-uncased,Base,127.039289,119.737864,104.492188,150.061131,0.809167,0.762662,104.492188,150.061131,5,"[117.91324615478516, 150.06113052368164, 104.4...",157,Hoping to rebound from their loss to the Patr...,Which team won the game?,2c7c93f6-69ed-47cc-a5af-94a00c185a26
3,3,drop,bert-base-uncased,Base,119.862890,109.892130,100.628376,161.371231,0.763458,0.699950,100.628376,161.371231,5,"[119.31896209716795, 161.37123107910156, 109.8...",157,Hoping to rebound from their loss to the Patr...,How many field goals did both teams kick in th...,7dfd2b64-f39e-4bb4-aeb0-1900adda6018
4,4,drop,bert-base-uncased,Base,111.079550,109.916925,101.395845,126.971960,0.707513,0.700108,101.395845,126.971960,5,"[126.97196006774902, 113.73090744018556, 103.3...",157,Hoping to rebound from their loss to the Patr...,How many more yards was Kris Browns's first fi...,121a8f57-7752-4373-a9ba-748b2c577cd2


In [170]:
df["time_per_token"] = df["time once (ms)"]/df["seq_length"]

KeyError: 'time once (ms)'

In [140]:
print(len(df))
# df.head()
df.tail()


24000


,reader,adapter,model_name,time once (ms),average_time 50 times (ms),seq_length,context,question,data_id,time_per_token
23995,roberta-base,squad_v2,ONNX-OPT Quantized,43.093204,NaN,96,"In 1066, Duke William II of Normandy conquered...",Where did Harold II die?,56de16ca4396321400ee25c5,0.448888
23996,roberta-base,squad_v2,ONNX-OPT Quantized,38.706064,NaN,96,"In 1066, Duke William II of Normandy conquered...",Who killed Harold II?,56de16ca4396321400ee25c6,0.403188
23997,roberta-base,squad_v2,ONNX-OPT Quantized,37.344933,NaN,96,"In 1066, Duke William II of Normandy conquered...",When was the Battle of Hastings?,56de16ca4396321400ee25c7,0.389010
23998,roberta-base,squad_v2,ONNX-OPT Quantized,40.225029,NaN,96,"In 1066, Duke William II of Normandy conquered...",Who was the ruling class ahead of the Normans?,56de16ca4396321400ee25c8,0.419011
23999,roberta-base,squad_v2,ONNX-OPT Quantized,37.925959,NaN,96,"In 1066, Duke William II of Normandy conquered...",When did King Harold II conquer England?,5ad3f4b1604f3c001a3ff951,0.395062


In [141]:
data_id_list = df["data_id"].unique().tolist()
adapter_list = df["adapter"].unique().tolist()
reader_list = df["reader"].unique().tolist()
model_name_list = df["model_name"].unique().tolist()

In [165]:
df_fin = pd.DataFrame(columns=["adapter", "reader", "model_name", "mean_time","median_time", "min_time", "max_time", "mean_time_per_token", "median_time_per_token", "min_time_per_token", "max_time_per_token", "runs", "time_unique_values", "seq_length", "context", "question", "data_id"])
df_overall = pd.DataFrame(columns=["adapter", "reader", "model_name", "mean_time","median_time", "min_time", "max_time", "mean_time_per_token", "median_time_per_token", "min_time_per_token", "max_time_per_token", "runs", "av_seq_length"])

In [167]:
for adapter in adapter_list:
    # print(f"Doing {adapter}")
    df_adapter = df[df["adapter"] == adapter]
    
    for reader in reader_list:
        # print(f"Doing {reader}")
        df_reader = df_adapter[df_adapter["reader"] == reader]
        
        for model_name in model_name_list:
            # print(f"Doing {model_name}")
            df_model = df_reader[df_reader["model_name"]== model_name]

            for data_id in data_id_list:
                df_data_id = df_model[df_model["data_id"] == data_id]
                if df_data_id.empty:
                    continue
                # print(f"Doing {data_id}")
                
                time_unique_values = df_data_id["time once (ms)"].tolist()
                runs = len(time_unique_values)

                mean_time = df_data_id["time once (ms)"].mean()
                median_time = df_data_id["time once (ms)"].median()
                min_time = df_data_id["time once (ms)"].min()
                max_time = df_data_id["time once (ms)"].max()
                
                mean_time_per_token = df_data_id["time_per_token"].mean()
                median_time_per_token = df_data_id["time_per_token"].median()
                min_time_per_token = df_data_id["time once (ms)"].min()
                max_time_per_token = df_data_id["time once (ms)"].max()

                # add question, context. 
                seq_length = df_data_id["seq_length"].unique()[0]
                context = df_data_id["context"].unique()[0]
                question = df_data_id["question"].unique()[0]


                df_fin.loc[len(df_fin)] = [adapter, reader, model_name, mean_time,median_time, min_time, max_time, mean_time_per_token, median_time_per_token, min_time_per_token, max_time_per_token, runs, time_unique_values, seq_length, context, question, data_id]

            

            runs = len(df_model["time once (ms)"].tolist())

            overall_mean_time = df_model["time once (ms)"].mean()
            overall_median_time = df_model["time once (ms)"].median()
            overall_min_time = df_model["time once (ms)"].min()
            overall_max_time = df_model["time once (ms)"].max()

            overall_mean_time_per_token = df_model["time_per_token"].mean()
            overall_median_time_per_token = df_model["time_per_token"].median()
            overall_min_time_per_token = df_model["time once (ms)"].min()
            overall_max_time_per_token = df_model["time once (ms)"].max()
            av_seq_length = df_model["seq_length"].sum()/len(df_model["seq_length"])
            
            df_overall.loc[len(df_fin)] = [adapter, reader, model_name, mean_time, median_time, min_time, max_time, mean_time_per_token, median_time_per_token, min_time_per_token, max_time_per_token, runs, av_seq_length]

In [168]:
df_overall.to_csv(f"inference_time_{skill}_overall.csv")
df_overall.to_excel(f"inference_time_{skill}_overall.xlsx")

In [158]:
df_fin.to_csv(f"inference_time_{skill}_overall.csv")
df_fin.to_excel(f"inference_time_{skill}_overall.xlsx") # drop duplicates - currently manually

### archive

In [43]:
for data_id in df["data_id"]:
    print(len(df))

    rows = df.loc[df["data_id"] == data_id]

    for model_name in df['model_name'].unique():
        
        model = rows[rows["model_name"] == model_name]
        
        try: 
            runs = len(model)

            run_min = model["time once (ms)"].min()
            run_max = model["time once (ms)"].max()
            run_median = model["time once (ms)"].median()

            seq_length = model["seq_length"].values[0]

            min_time_per_token = run_min/seq_length
            max_time_per_token = run_max/seq_length
            median_time_per_token = run_median/seq_length
            
            context = model["context"].values[0]
            question = model["question"].values[0]
        

            df_fin.loc[len(df_fin)] = [model_name, "", run_min, run_max, run_median, runs, seq_length, min_time_per_token, max_time_per_token, median_time_per_token, context, question, data_id]
            
        except Exception as e:
            print("error")
            print(e)
            pass

        break
    break

    
    
    df = df.drop(df.loc[df["data_id"] == data_id].index) #reduce search space


4200


In [44]:
model

,model_name,time once (ms),average_time 50 times (ms),seq_length,context,question,choices,data_id,data_set_name,time_per_token
0,Base,258.271933,NaN,92,Do i need to go for a legal divorce ? I wanted...,Why is this person asking about divorce ?,['If he gets married in the church he wo nt ha...,0,cosmos_qa,2.807304
600,Base,299.651861,NaN,92,Do i need to go for a legal divorce ? I wanted...,Why is this person asking about divorce ?,['If he gets married in the church he wo nt ha...,0,cosmos_qa,3.257085
1200,Base,1240.674257,NaN,310,Candy watched the bearded man drive his silver...,How long was Candy trying to seduce Larry?,"['about 10 minutes', 'about 2 hours', 'not eno...",0,quail,4.002175
1800,Base,1662.490129,NaN,310,Candy watched the bearded man drive his silver...,How long was Candy trying to seduce Larry?,"['about 10 minutes', 'about 2 hours', 'not eno...",0,quail,5.362871
2400,Base,62.294006,NaN,19,"When particles of matter are closer together, ...",If Jim moves some particles of matter farther ...,"['decrease', 'increase']",0,quartz,3.278632
3000,Base,1608.655930,NaN,339,"I am a psychologist. I first met Timothy, a qu...",What did the writer think of Timothy after lea...,"['Timothy was very hardworking.', 'Timothy was...",0,race,4.745298
3600,Base,1163.430929,NaN,339,"I am a psychologist. I first met Timothy, a qu...",What did the writer think of Timothy after lea...,"['Timothy was very hardworking.', 'Timothy was...",0,race,3.431950


In [8]:
df_fin.to_csv("inference_time_categorical.csv")
df_fin.to_excel("inference_time_categorical.xlsx") # drop duplicates - currently manually

In [97]:
print(len(df_fin))

274708


In [100]:
df_droped = df_fin.drop_duplicates()

In [101]:
df_droped.to_csv("analyse4.csv")
df_droped.to_excel("analyse4.xlsx")